In [1]:
import backtrader as bt
import pandas as pd
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
from collections import deque
from tabulate import tabulate
st_date = datetime.datetime(2023, 5, 4)
ed_date = datetime.datetime(2023, 6, 30)

tf = pd.read_csv('tradeinfo.csv',parse_dates = True,infer_datetime_format=True,header=None)
tf.columns = ['datetime','dir','ct','price','lot','dd','tp','info','g','ddd']
tf = tf.set_index('datetime')
tf.index = pd.to_datetime(tf.index,format='%Y-%m-%d_%H:%M:%S.%f')

tf = tf.loc['2023-05-04':'2023-06-30']
tradelist = []
for idx,r in tf.iterrows():
    # print(r)
    ct = 'CN'
    # print(r['dir'])
    if 'IF' in r['ct']:
        ct = 'IF'
    if 'IC' in r['ct']:
        ct = 'IC'
    size = r['lot'] if r['dir'] == 'buy' else -r['lot']
    tradelist.append(
                        (
                        idx,#datetime.datetime.strftime(idx,'%Y-%m-%d %H:%M:%S'),
                        size,
                        r['price'],
                        ct
                        )
                    )

C:\Users\siruo\AppData\Local\Temp\ipykernel_8108\1118497575.py:23: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  tf = tf.loc['2023-5-4':'2023-06-17']


In [2]:
dividend_cn = pd.read_excel("Data/股利.xlsx",sheet_name=3)[['更新时间','主力股利']]
dividend_ih = pd.read_excel("Data/股利.xlsx",sheet_name=4)[['更新时间','主力股利']]
dividend_if = pd.read_excel("Data/股利.xlsx",sheet_name=5)[['更新时间','主力股利']]
dividend_ic = pd.read_excel("Data/股利.xlsx",sheet_name=6)[['更新时间','主力股利']]

dividend_cn.columns = ['Time','Dividends']
dividend_ih.columns = ['Dates','Dividends']
dividend_if.columns = ['Dates','Dividends']
dividend_ic.columns = ['Dates','Dividends']

a50 = pd.read_csv('Data/Index/A50.csv',skiprows = [0,1])
a50['Time'] = pd.to_datetime(a50['Dates']).dt.floor('d')
a50 = pd.merge(a50, dividend_cn, on='Time', how="left")
a50['Close'] = a50['Close'] - a50['Dividends']
a50.drop(columns=['Time','Dividends'], inplace=True)

csi300 = pd.read_csv('Data/Index/CSI300.csv',skiprows = [0])
csi300['Dates'] = pd.to_datetime(csi300['time']).dt.floor('d')
csi300 = pd.merge(csi300, dividend_if, on='Dates', how="left")
csi300['close'] = csi300['close'] - csi300['Dividends']
csi300.drop(columns=['Dates','Dividends'], inplace=True)

csi500 = pd.read_csv('Data/Index/CSI500.csv',skiprows = [0])
csi500['Dates'] = pd.to_datetime(csi500['time']).dt.floor('d')
csi500 = pd.merge(csi500, dividend_ic, on='Dates', how="left")
csi500['close'] = csi500['close'] - csi500['Dividends']
csi500.drop(columns=['Dates','Dividends'], inplace=True)

sse50 = pd.read_csv('Data/Index/SSE50.csv',skiprows = [0])
sse50["Dates"] = pd.to_datetime(sse50['time']).dt.floor('d')
sse50 = pd.merge(sse50, dividend_ih, on='Dates', how="left")
sse50['close'] = sse50['close'] - sse50['Dividends']
sse50.drop(columns=['Dates','Dividends'], inplace=True)

In [3]:
a50.to_csv("Data/Index/A50(1).csv",index=False)
csi300.to_csv("Data/Index/CSI300(1).csv",index=False)
csi500.to_csv("Data/Index/CSI500(1).csv",index=False)
sse50.to_csv("Data/Index/SSE50(1).csv",index=False)

In [4]:
if_codes = [
            # 'IF1910','IF1911','IF1912',
            # 'IF2001','IF2002','IF2003','IF2004','IF2005','IF2006',
            # 'IF2007','IF2008','IF2009','IF2010','IF2011','IF2012',
            # 'IF2101','IF2102','IF2103','IF2104','IF2105','IF2106',
            # 'IF2107','IF2108','IF2109','IF2110','IF2111','IF2112',
            # 'IF2201','IF2202','IF2203','IF2204','IF2205','IF2206',
            # 'IF2207','IF2208','IF2209','IF2210','IF2211','IF2212',
            # 'IF2301','IF2302'
            'IF2303','IF2304','IF2305','IF2306','IF2307'
            ]
ic_codes = [
            # 'IC1910','IC1911','IC1912',
            # 'IC2001','IC2002','IC2003','IC2004','IC2005','IC2006',
            # 'IC2007','IC2008','IC2009','IC2010','IC2011','IC2012',
            # 'IC2101','IC2102','IC2103','IC2104','IC2105','IC2106',
            # 'IC2107','IC2108','IC2109','IC2110','IC2111','IC2112',
            # 'IC2201','IC2202','IC2203','IC2204','IC2205','IC2206',
            # 'IC2207','IC2208','IC2209','IC2210','IC2211','IC2212',
            # 'IC2301','IC2302'
            'IC2303','IC2304','IC2305','IC2306','IC2307'
            ]
ih_codes = [
            # 'IH1910','IH1911','IH1912',
            # 'IH2001','IH2002','IH2003','IH2004','IH2005','IH2006',
            # 'IH2007','IH2008','IH2009','IH2010','IH2011','IH2012',
            # 'IH2101','IH2102','IH2103','IH2104','IH2105','IH2106',
            # 'IH2107','IH2108','IH2109','IH2110','IH2111','IH2112',
            #'IH2201','IH2202','IH2203','IH2204','IH2205','IH2206',
            'IH2207']
cn_codes =  [
            # 'CN2111','CN2112',
            # 'CN2201','CN2202','CN2203','CN2204','CN2205','CN2206',
            # 'CN2301','CN2302'
            'CN2303', 'CN2304','CN2305','CN2306','CN2307'
            ]
if_data = {}
ic_data = {}
ih_data = {}
cn_data = {}


for a in if_codes:
    if_data[a] =  pd.read_csv('Data/IF/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True,encoding='ISO-8859-1').dropna()
for a in ic_codes:
    ic_data[a] =  pd.read_csv('Data/IC/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True,encoding='ISO-8859-1').dropna()
for a in ih_codes:
    ih_data[a] =  pd.read_csv('Data/IH/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True,encoding='ISO-8859-1').dropna()
for a in cn_codes:
    cn_data[a] =  pd.read_csv('Data/CN/'+a+'.csv', index_col = 0,skiprows = [0,2],parse_dates=True,encoding='ISO-8859-1').dropna()

csi300 = pd.read_csv('Data/Index/CSI300(1).csv', index_col = 0,parse_dates=True,encoding='ISO-8859-1')
csi500 = pd.read_csv('Data/Index/CSI500(1).csv',index_col = 0,parse_dates=True,encoding='ISO-8859-1')
sse50 = pd.read_csv('Data/Index/SSE50(1).csv',index_col = 0,parse_dates=True,encoding='ISO-8859-1')
a50 = pd.read_csv('Data/Index/A50(1).csv',index_col = 0,parse_dates=True,encoding='ISO-8859-1')

In [5]:
if_feeds = []
ic_feeds = []
cn_feeds = []
ih_feeds = []

if_rolldate = []
ic_rolldate = []
ih_rolldate = []

cn_rolldate = []

for k in if_data.keys():
    if if_data[k].index[-1] < pd.to_datetime(st_date):
        # print(if_data[k].index[-1])
        continue
    if_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    if_data[k]['Volume'] = if_data[k]['Volume']*100000
    if_data[k] = if_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    if_data[k] = if_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    if_feeds.append(bt.feeds.PandasData(dataname = if_data[k],name = k))
    if_rolldate.append(if_data[k].resample('1d').last().index[-2]+datetime.timedelta(hours=14,minutes=57))

for k in ic_data.keys():
    if ic_data[k].index[-1] < pd.to_datetime(st_date):
        # print(ic_data[k].index[-1])
        continue
    ic_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    ic_data[k]['Volume'] = ic_data[k]['Volume']*100000
    ic_data[k] = ic_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    ic_data[k] = ic_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    ic_feeds.append(bt.feeds.PandasData(dataname = ic_data[k],name = k))

for k in ih_data.keys():
    if ih_data[k].index[-1] < pd.to_datetime(st_date):
        # print(ih_data[k].index[-1])
        continue
    ih_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    ih_data[k]['Volume'] = ih_data[k]['Volume']*100000
    ih_data[k] = ih_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    ih_data[k] = ih_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()

    ih_feeds.append(bt.feeds.PandasData(dataname = ih_data[k],name = k))



csi300 = csi300.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
csi300.fillna(inplace = True,method = 'ffill')
csi500 = csi500.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
csi500.fillna(inplace = True,method = 'ffill')

sse50 = sse50.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
sse50.fillna(inplace = True,method = 'ffill')
a50 = a50.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index().drop_duplicates()
a50.fillna(inplace = True,method = 'ffill')

for k in cn_data.keys():    
    # cn_data[k]
    if cn_data[k].index[-1] < pd.to_datetime(st_date):
        # print(cn_data[k].index[-1])
        continue

    cn_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    cn_data[k]['Volume'] = cn_data[k]['Volume']*100000
    cn_data[k] = cn_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    cn_data[k] = cn_data[k].fillna(method= 'ffill')
    cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
    cn_feeds.append(bt.feeds.PandasData(dataname = cn_data[k],name = k))
    cn_rolldate.append(cn_data[k].resample('1d').last().index[-1]+datetime.timedelta(hours=14,minutes=57))

csi300_feed = bt.feeds.PandasData(dataname = csi300,fromdate=st_date,todate=ed_date)
csi500_feed = bt.feeds.PandasData(dataname = csi500,fromdate=st_date,todate=ed_date)
# sse50_feed = bt.feeds.PandasData(dataname = sse50,fromdate=st_date,todate=ed_date)

a50_feed = bt.feeds.PandasData(dataname = a50,fromdate=st_date,todate=ed_date)
if_rolldate = if_rolldate[:-1]
# ic_rolldate = if_rolldate
cn_rolldate = cn_rolldate[:-1]

def cn_expire(dt,d):
    return dt in cn_rolldate

def if_expire(dt,d):
    if dt in if_rolldate :
        return True
    return False



C:\Users\siruo\AppData\Local\Temp\ipykernel_8108\4079463248.py:68: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
C:\Users\siruo\AppData\Local\Temp\ipykernel_8108\4079463248.py:68: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cn_data[k] = cn_data[k].loc[cn_data[k].index&a50.index].dropna().sort_index().drop_duplicates()
C:\Users\siruo\AppData\Local\Temp\ipykernel_8108\4079463248.py:68: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cn_data[k] = cn_data[k].loc[cn_data[k].index&a

In [6]:
a50

,Open,Close,High,Low,Volume,Oi
Dates,,,,,,
2023-05-04 09:30:00,13163.02,13114.95,13163.02,13128.35,0,NaN
2023-05-04 09:31:00,13137.90,13107.29,13139.87,13127.90,0,NaN
2023-05-04 09:32:00,13129.96,13092.04,13131.38,13111.88,0,NaN
2023-05-04 09:33:00,13115.21,13081.08,13116.69,13099.69,0,NaN
2023-05-04 09:34:00,13104.12,13071.05,13105.15,13094.02,0,NaN
...,...,...,...,...,...,...
2023-06-16 14:53:00,13052.11,13012.98,13053.02,13047.00,0,NaN
2023-06-16 14:54:00,13048.92,13010.32,13051.37,13045.02,0,NaN
2023-06-16 14:55:00,13046.15,13011.37,13048.21,13043.95,0,NaN


In [7]:
class IFICStrategy(bt.Strategy):
    params = (

        ('margin', 20),  
        ('distor', 3),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',28),
        ('logname','log.csv'),

    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close[0],
                                            self.csi500_close[0],
                                            self.a50_close[0],
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    

    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                       
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Datetime,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = self.data3.close
        self.csi500_close = self.data4.close
        self.a50_close = self.data5.close
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        
    def next(self):
        
        self.cn_theo = self.a50_close[0]*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*(self.csi300_close[0])-self.p.ic_lot*2*(self.csi500_close[0]))
        # print(self.a50_close[0],self.csi300_close[0],self.csi500_close[0],self.if_close[0],self.ic_close[0])
        cn_bid = self.cn_theo - self.p.margin + self.base
        cn_ask = self.cn_theo + self.p.margin + self.base
        self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],self.csi300_close[0],self.csi500_close[0],self.a50_close[0],self.broker.get_value()))
        self.write_record()
        if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
            size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
            self.buy( data = self.getdatabyname('IF'),
                      size = self.p.if_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('IC'),
                       size = self.p.ic_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('CN'),
                       size = self.p.cn_lot*size, # size < 0
                       exectype=bt.Order.Market,coc=True)
            self.base = self.base + size * self.p.distor # base 上移，抬高理论价
            
        if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
            size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
            self.sell( data = self.getdatabyname('IF'),
                       size = self.p.if_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('IC'),
                      size = self.p.ic_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('CN'),
                      size = self.p.cn_lot*size, # size < 0
                      exectype=bt.Order.Market,coc=True)
            self.base = self.base - size * self.p.distor

        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
            print('ROLL')
            self.base = 0

            if self.getpositionbyname('IF').size != 0:
                self.order_target_size( data = self.getdatabyname('IF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('IC').size != 0:
                self.order_target_size( data = self.getdatabyname('IC'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
            print('ROLL')
            self.base = 0
            if self.getpositionbyname('IF').size != 0:
                self.order_target_size( data = self.getdatabyname('IF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('IC').size != 0:
                self.order_target_size( data = self.getdatabyname('IC'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('CN').size != 0:
                self.order_target_size( data = self.getdatabyname('CN'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)

    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [8]:
class IFICAbsDailyOpenStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',27),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            # if self.datetime.time() == datetime.time(hour = 9, minute=30):
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.datetime.time() == datetime.time(hour = 9, minute=30): #and self.datetime.date().isocalendar()[1] >= self.week+4:
                self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*(self.csi300_close)-self.p.ic_lot*2*(self.csi500_close))
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [9]:
class IFICAbsDailyCloseStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            # if self.datetime.time() == datetime.time(hour = 9, minute=30):
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.datetime.time() == datetime.time(hour = 14, minute=59): #and self.datetime.date().isocalendar()[1] >= self.week+4:
                print('REbasing!')
                self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*(self.csi300_close)-self.p.ic_lot*2*(self.csi500_close))
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [10]:
class IFICAbsRollStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
        
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.data.datetime.datetime() in if_rolldate:# or self.data.datetime.datetime() in cn_rolldate:#and self.datetime.date().isocalendar()[1] >= self.week+4:
                # self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*(self.csi300_close)-self.p.ic_lot*2*(self.csi500_close))
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            if self.cn_close > cn_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出CN
                size = (self.cn_close - cn_ask)//self.p.distor + 1 # size > 0
                self.buy( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.sell( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base + size * self.p.distor # base 上移，抬高理论价
                
            if self.cn_close < cn_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
                size = (cn_bid - self.cn_close)//self.p.distor + 1 # size > 0
                self.sell( data = self.getdatabyname('IF'),
                        size = self.p.if_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('IC'),
                        size = self.p.ic_lot*size,
                        exectype=bt.Order.Market,coc=True)
                self.buy( data = self.getdatabyname('CN'),
                        size = self.p.cn_lot*size, # size < 0
                        exectype=bt.Order.Market,coc=True)
                self.base = self.base - size * self.p.distor

            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
                self.base = 0

                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
            if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
                self.base = 0
                if self.getpositionbyname('IF').size != 0:
                    self.order_target_size( data = self.getdatabyname('IF'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('IC').size != 0:
                    self.order_target_size( data = self.getdatabyname('IC'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)
                if self.getpositionbyname('CN').size != 0:
                    self.order_target_size( data = self.getdatabyname('CN'),
                                            target= 0,
                                            exectype=bt.Order.Market,
                                            coc = True)


    # def next_open(self):
        # 换月重新开仓
    #     # if self.data.datetime.datetime()in if_rolldate:  
    #             self.buy(data=self.getdatabyname('IF'),exectype=bt.Order.Market,coo = True)
    #             self.buy(data=self.getdatabyname('IC'),exectype=bt.Order.Market,coo = True)
    #     if self.data.datetime.datetime()in cn_rolldate:  
    #             self.buy(data=self.getdatabyname('CN'),exectype=bt.Order.Market,coo = True)
    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [11]:
class IFICReplayStrategy(bt.Strategy):
    params = (

        ('margin', 25),  
        ('distor', 5),
        ('bias', 0),
        ('if_lot',3),
        ('ic_lot',1),
        ('cn_lot',30),
        ('logname','abs.csv'),

    )
    def start(self):
        header = [ 'len','Name','Code', 'Datetime', 'WeekDay', 'Open',
                  'High', 'Low', 'Close', 'Volume', 'OpenInterest']
        print(', '.join(header))

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f,%d, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.csi300_close,
                                            self.csi500_close,
                                            self.a50_close,
                                            self.if_close[0],
                                            self.ic_close[0],
                                            self.cn_close[0],
                                            self.getpositionbyname('IF').size,
                                            self.getpositionbyname('IC').size,
                                            self.getpositionbyname('CN').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Dates,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.if_close = self.data0.close
        self.ic_close = self.data1.close
        self.cn_close = self.data2.close
        self.csi300_close = None
        self.csi500_close = None
        self.a50_close = None
        self.cn_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        self.week = 0

    def next(self):
        if self.csi300_close is None or self.csi500_close is None or self.a50_close is None:
            self.csi300_close = self.data3.close[0]
            self.csi500_close = self.data4.close[0]
            self.a50_close = self.data5.close[0]
            # self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()
        else:
            # print(self.datetime.date().weekday())
            
            if self.data.datetime.datetime() in if_rolldate:# or self.data.datetime.datetime() in cn_rolldate:#and self.datetime.date().isocalendar()[1] >= self.week+4:
                # self.week = self.datetime.date().isocalendar()[1]
                self.csi300_close = self.data3.close[0]
                self.csi500_close = self.data4.close[0]
                self.a50_close = self.data5.close[0]
            self.cn_theo = self.a50_close*(self.p.if_lot*3*self.if_close[0] - self.p.ic_lot*2*self.ic_close[0])/(self.p.if_lot*3*(self.csi300_close)-self.p.ic_lot*2*(self.csi500_close))
            cn_bid = self.cn_theo - self.p.margin + self.base
            cn_ask = self.cn_theo + self.p.margin + self.base
            self.log("%.2f,%.2f,%.2f,%.2f,%.2f,%.d,%.d"%(self.if_close[0],self.ic_close[0],self.cn_close[0],cn_ask,cn_bid,self.base,self.broker.get_value()))
            self.write_record()

            

            # if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in if_rolldate:
            #     self.base = 0

            #     if self.getpositionbyname('IF').size != 0:
            #         self.order_target_size( data = self.getdatabyname('IF'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)
            #     if self.getpositionbyname('IC').size != 0:
            #         self.order_target_size( data = self.getdatabyname('IC'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)
            #     if self.getpositionbyname('CN').size != 0:
            #         self.order_target_size( data = self.getdatabyname('CN'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)
            # if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in cn_rolldate:
            #     self.base = 0
            #     if self.getpositionbyname('IF').size != 0:
            #         self.order_target_size( data = self.getdatabyname('IF'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)
            #     if self.getpositionbyname('IC').size != 0:
            #         self.order_target_size( data = self.getdatabyname('IC'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)
            #     if self.getpositionbyname('CN').size != 0:
            #         self.order_target_size( data = self.getdatabyname('CN'),
            #                                 target= 0,
            #                                 exectype=bt.Order.Market,
            #                                 coc = True)


    # def next_open(self):
        # 换月重新开仓
    #     # if self.data.datetime.datetime()in if_rolldate:  
    #             self.buy(data=self.getdatabyname('IF'),exectype=bt.Order.Market,coo = True)
    #             self.buy(data=self.getdatabyname('IC'),exectype=bt.Order.Market,coo = True)
    #     if self.data.datetime.datetime()in cn_rolldate:  
    #             self.buy(data=self.getdatabyname('CN'),exectype=bt.Order.Market,coo = True)
    def stop(self):
        self.tradeLog.close()
        self.portlog.close()
                


In [12]:
from tabulate import tabulate

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(100000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='IF', *if_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='IC', *ic_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='CN', *cn_feeds,checkdate = cn_expire,fromdate=st_date,todate=ed_date)
cerebro.adddata(csi300_feed)
cerebro.adddata(csi500_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=300,
                             margin=150000,
                             name='IF')
cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=200,
                             margin=160000,
                             name='IC')
cerebro.broker.setcommission(commission=12.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=6.8,
                             margin=10000,
                             name='CN')


cerebro.addstrategy(IFICStrategy,if_lot = 3,cn_lot = 28,logname = '3IF_IC_28CN_Basis.csv')



strats = cerebro.run(runonce = False,cheat_on_open=True,tradehistory = True)


0

2023-05-04T09:30:00, 4020.80,6246.20,13152.00,4012.35,6225.20,13114,100000000
2023-05-04T09:31:00, 4015.60,6248.60,13134.00,4008.77,6230.92,13107,100000000
2023-05-04T09:32:00, 4011.60,6247.00,13124.00,4002.68,6224.65,13092,100000000
2023-05-04T09:33:00, 4006.40,6239.60,13102.00,4000.72,6217.85,13081,100000000
2023-05-04T09:34:00, 4009.40,6245.60,13105.00,4002.48,6227.52,13071,100000000
2023-05-04T09:35:00, 4013.40,6248.80,13128.00,4008.25,6239.58,13097,100000000
2023-05-04T09:36:00, 4012.00,6248.60,13125.00,4006.59,6236.40,13091,100000000
2023-05-04T09:37:00, BUY EXECUTED,IF,4012.00,3,0.00,3,90,1,99999086.80

2023-05-04T09:37:00, SELL EXECUTED,IC,6248.60,-1,0.00,-1,30,2,99999086.80

2023-05-04T09:37:00, SELL EXECUTED,CN,13125.00,-28,0.00,-28,350,3,99999086.80

2023-05-04T09:37:00, 4012.80,6246.80,13133.00,4008.46,6237.50,13099,99999086
2023-05-04T09:38:00, 4011.40,6246.40,13120.00,4009.57,6238.53,13093,100000382
2023-05-04T09:39:00, BUY EXECUTED,IF,4011.40,3,0.00,6,90,4,100001348.80



In [13]:

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(100000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='IF', *if_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='IC', *ic_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date,)
cerebro.rolloverdata(name='CN', *cn_feeds,checkdate = cn_expire,fromdate=st_date,todate=ed_date,)
cerebro.adddata(csi300_feed)
cerebro.adddata(csi500_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=300,
                             margin=150000,
                             name='IF')
cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=200,
                             margin=160000,
                             name='IC')
cerebro.broker.setcommission(commission=12.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=6.8,
                             margin=10000,
                             name='CN')
cerebro.addstrategy(IFICReplayStrategy,if_lot = 3,cn_lot = 28,ic_lot = 1,logname = 'replay.csv')
cerebro.add_order_history(tradelist, notify=True)


strats = cerebro.run(runonce = False,cheat_on_open=True,tradehistory = True)

0

len, Name, Code, Datetime, WeekDay, Open, High, Low, Close, Volume, OpenInterest
2023-05-04T09:31:00, 4015.60,6248.60,13134.00,13130.22,13080.22,0,100000000
2023-05-04T09:32:00, 4011.60,6247.00,13124.00,13112.04,13062.04,0,100000000
2023-05-04T09:33:00, 4006.40,6239.60,13102.00,13094.30,13044.30,0,100000000
2023-05-04T09:34:00, 4009.40,6245.60,13105.00,13102.62,13052.62,0,100000000
2023-05-04T09:35:00, 4013.40,6248.80,13128.00,13119.03,13069.03,0,100000000
2023-05-04T09:36:00, 4012.00,6248.60,13125.00,13112.26,13062.26,0,100000000
2023-05-04T09:37:00, 4012.80,6246.80,13133.00,13118.25,13068.25,0,100000000
2023-05-04T09:38:00, 4011.40,6246.40,13120.00,13111.71,13061.71,0,100000000
2023-05-04T09:39:00, 4013.40,6248.00,13124.00,13119.91,13069.91,0,100000000
2023-05-04T09:40:00, 4011.40,6248.80,13114.00,13109.05,13059.05,0,100000000
2023-05-04T09:41:00, 4011.80,6249.20,13112.00,13110.60,13060.60,0,100000000
2023-05-04T09:42:00, 4010.80,6248.00,13107.00,13106.94,13056.94,0,100000000
2023-05

In [14]:

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(100000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='IF', *if_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='IC', *ic_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date,)
cerebro.rolloverdata(name='CN', *cn_feeds,checkdate = cn_expire,fromdate=st_date,todate=ed_date,)
cerebro.adddata(csi300_feed)
cerebro.adddata(csi500_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=300,
                             margin=150000,
                             name='IF')
cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=200,
                             margin=160000,
                             name='IC')
cerebro.broker.setcommission(commission=12.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=6.8,
                             margin=10000,
                             name='CN')

cerebro.addstrategy(IFICAbsDailyOpenStrategy,if_lot = 3,cn_lot = 28,logname = '3IF_IC_28CN_DailyO.csv')


strats = cerebro.run(runonce = False,cheat_on_open=True,tradehistory = True)

0

len, Name, Code, Datetime, WeekDay, Open, High, Low, Close, Volume, OpenInterest
2023-05-04T09:31:00, 4015.60,6248.60,13134.00,13130.22,13080.22,0,100000000
2023-05-04T09:32:00, BUY EXECUTED,IF,2023-05-04 09:31:00,4015.60,3,0.00,3,90,1566,99998154.00

2023-05-04T09:32:00, SELL EXECUTED,IC,2023-05-04 09:31:00,6248.60,-1,0.00,-1,30,1567,99998154.00

2023-05-04T09:32:00, SELL EXECUTED,CN,2023-05-04 09:31:00,13134.00,-28,0.00,-28,350,1568,99998154.00

2023-05-04T09:32:00, 4011.60,6247.00,13124.00,13117.04,13067.04,5,99998154
2023-05-04T09:33:00, BUY EXECUTED,IF,2023-05-04 09:32:00,4011.60,6,0.00,9,180,1569,100000180.40

2023-05-04T09:33:00, SELL EXECUTED,IC,2023-05-04 09:32:00,6247.00,-2,0.00,-3,60,1570,100000180.40

2023-05-04T09:33:00, SELL EXECUTED,CN,2023-05-04 09:32:00,13124.00,-56,0.00,-84,700,1571,100000180.40

2023-05-04T09:33:00, 4006.40,6239.60,13102.00,13109.30,13059.30,15,100000180
2023-05-04T09:34:00, 4009.40,6245.60,13105.00,13117.62,13067.62,15,100002966
2023-05-04T09:35:00,

In [15]:

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(100000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='IF', *if_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date)
cerebro.rolloverdata(name='IC', *ic_feeds,checkdate = if_expire,fromdate=st_date,todate=ed_date,)
cerebro.rolloverdata(name='CN', *cn_feeds,checkdate = cn_expire,fromdate=st_date,todate=ed_date,)
cerebro.adddata(csi300_feed)
cerebro.adddata(csi500_feed)
cerebro.adddata(a50_feed)

cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=300,
                             margin=150000,
                             name='IF')
cerebro.broker.setcommission(commission=30,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=200,
                             margin=160000,
                             name='IC')
cerebro.broker.setcommission(commission=12.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=6.8,
                             margin=10000,
                             name='CN')

cerebro.addstrategy(IFICAbsRollStrategy,if_lot = 3,cn_lot = 28,logname = '3IF_IC_28CN_Roll.csv')


strats = cerebro.run(runonce = False,cheat_on_open=True,tradehistory = True)

0

len, Name, Code, Datetime, WeekDay, Open, High, Low, Close, Volume, OpenInterest
2023-05-04T09:31:00, 4015.60,6248.60,13134.00,13130.22,13080.22,0,100000000
2023-05-04T09:32:00, BUY EXECUTED,IF,2023-05-04 09:31:00,4015.60,3,0.00,3,90,2109,99998154.00

2023-05-04T09:32:00, SELL EXECUTED,IC,2023-05-04 09:31:00,6248.60,-1,0.00,-1,30,2110,99998154.00

2023-05-04T09:32:00, SELL EXECUTED,CN,2023-05-04 09:31:00,13134.00,-28,0.00,-28,350,2111,99998154.00

2023-05-04T09:32:00, 4011.60,6247.00,13124.00,13117.04,13067.04,5,99998154
2023-05-04T09:33:00, BUY EXECUTED,IF,2023-05-04 09:32:00,4011.60,6,0.00,9,180,2112,100000180.40

2023-05-04T09:33:00, SELL EXECUTED,IC,2023-05-04 09:32:00,6247.00,-2,0.00,-3,60,2113,100000180.40

2023-05-04T09:33:00, SELL EXECUTED,CN,2023-05-04 09:32:00,13124.00,-56,0.00,-84,700,2114,100000180.40

2023-05-04T09:33:00, 4006.40,6239.60,13102.00,13109.30,13059.30,15,100000180
2023-05-04T09:34:00, 4009.40,6245.60,13105.00,13117.62,13067.62,15,100002966
2023-05-04T09:35:00,

In [16]:
port_record_set = {}

port_record = pd.read_csv('portlog/3IF_IC_28CN_Basis.csv')
port_record['Dates'] = pd.to_datetime(port_record['Datetime']).dt.floor('d')
port_record = pd.merge(port_record, dividend_if, on='Dates', how="left")
port_record['CSI300'] = (port_record['CSI300'] + port_record['Dividends']).round(1)
port_record.drop(columns=['Dividends'], inplace=True)
port_record = pd.merge(port_record, dividend_ic, on='Dates', how="left")
port_record['CSI500'] = (port_record['CSI500'] + port_record['Dividends']).round(1)
port_record.drop(columns=['Dividends'], inplace=True)
port_record = pd.merge(port_record, dividend_cn, left_on='Dates', right_on='Time', how="left")
port_record['A50'] = (port_record['A50'] + port_record['Dividends']).round(1)
port_record.drop(columns=['Dates','Time','Dividends'], inplace=True)
# port_record.to_csv("portlog/3IF_IC_28CN_Basis(1).csv",index=False)

port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Basis'] = port_record


port_record = pd.read_csv('portlog/replay.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Replay'] = port_record


port_record = pd.read_csv('portlog/3IF_IC_28CN_DailyO.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['DailyOpen'] = port_record


port_record = pd.read_csv('portlog/3IF_IC_28CN_Roll.csv')
port_record = port_record[['Datetime','CSI300','CSI500','A50','IF','IC','CN','IF_pos','IC_pos','CN_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Roll'] = port_record

In [17]:
if_rolldate

[Timestamp('2023-05-18 14:57:00', freq='D'),
 Timestamp('2023-06-15 14:57:00', freq='D')]

In [18]:
port_record_set['Basis']

,CSI300,CSI500,A50,IF,IC,CN,IF_pos,IC_pos,CN_pos,Cash,Value
Datetime,,,,,,,,,,,
2023-05-04 09:30:00,4013.5,6237.2,13138.0,4020.8,6246.2,13152.0,0,0,0,100000000,100000000
2023-05-04 09:31:00,4010.0,6242.9,13130.3,4015.6,6248.6,13134.0,0,0,0,100000000,100000000
2023-05-04 09:32:00,4003.9,6236.6,13115.0,4011.6,6247.0,13124.0,0,0,0,100000000,100000000
2023-05-04 09:33:00,4001.9,6229.8,13104.1,4006.4,6239.6,13102.0,0,0,0,100000000,100000000
2023-05-04 09:34:00,4003.7,6239.5,13094.0,4009.4,6245.6,13105.0,0,0,0,100000000,100000000
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-16 14:55:00,3961.7,6147.3,13047.3,3931.4,6124.6,13017.0,-57,19,532,83159532,100069532
2023-06-16 14:56:00,3962.0,6148.2,13048.7,3932.0,6126.0,13016.0,-57,19,532,83150974,100060974
2023-06-16 14:57:00,3961.9,6148.8,13049.0,3932.4,6127.6,12858.0,0,0,0,100052044,100052044


In [20]:
# port_record = port_record.resample('5min').last().dropna()
totalPortValue = (
        Line(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis([a.isoformat(sep=' ') for a in port_record_set['Basis'].index.to_list()])
        .add_yaxis(
                    series_name="基差",
                    y_axis=round(port_record_set['Basis'].Value/10000).values.tolist()-port_record_set['Basis'].Value[0]/10000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width=1),
                    color="#88c4e8"
                    )
        .add_yaxis(
                    series_name="回放",
                    y_axis=round(port_record_set['Replay'].Value/10000).values.tolist()-port_record_set['Replay'].Value[0]/10000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width=1),
                    color="#fdc58f"
                    )
        .add_yaxis(
                    series_name="绝对值_DailyO",
                    y_axis=round(port_record_set['DailyOpen'].Value/10000).values.tolist()-port_record_set['DailyOpen'].Value[0]/10000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width=1),
                    color="#93cc82"
                    )
         .add_yaxis(
                    series_name="绝对值_换月调",
                    y_axis=round(port_record_set['Roll'].Value/10000).values.tolist()-port_record_set['Roll'].Value[0]/10000,
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width=1),
                    color="#ea9c9d"
                    )

        # .add_yaxis(
        #             series_name="绝对值_Roll",
        #             y_axis=round(port_record_set['Roll'].Value/10000).values.tolist()-port_record_set['Roll'].Value[0]/10000,
        #             is_symbol_show = False,
        #             label_opts=opts.LabelOpts(is_show=False),
        #             linestyle_opts=opts.LineStyleOpts(width = 1)
        # )
        # .add_yaxis(
        #             series_name="绝对值_RollN",
        #             y_axis=round(port_record_set['RollN'].Value/1000).values.tolist()-port_record_set['RollN'].Value[0]/1000,
        #             is_symbol_show = False,
        #             label_opts=opts.LabelOpts(is_show=False),
        #             linestyle_opts=opts.LineStyleOpts(width = 1)
        # # )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'CN Pos',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                    )
        
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            title_opts = opts.TitleOpts('3IF-IC-27CN',),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Value',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} W"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                                # opts.DataZoomOpts(
                                #     is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                                # ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )
totalCashUsage = (
    Bar()
    .add_xaxis([a.isoformat(sep=' ') for a in port_record.index.to_list()])
    .add_yaxis(
                series_name="基差",
                y_axis=port_record_set['Basis'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )
    .add_yaxis(
                series_name="回放",
                y_axis=port_record_set['Replay'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )
    # .add_yaxis(
    #             series_name="绝对值_Roll",
    #             y_axis=port_record_set['Roll'].CN_pos.values.tolist(),
    #             yaxis_index = 1,
    #             label_opts=opts.LabelOpts(is_show=False),
    #             itemstyle_opts = opts.ItemStyleOpts(opacity=0.05, )
    # )
    .add_yaxis(
                series_name="绝对值_DailyO",
                y_axis=port_record_set['DailyOpen'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )
    .add_yaxis(
                series_name="绝对值_换月调",
                y_axis=port_record_set['Roll'].CN_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )

   
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'position',
                                # max_ = 200,
                                # min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),

    )
)

totalPortValue.overlap(totalCashUsage)
totalPortValue.render('3ific.html')


'c:\\Users\\siruo\\Downloads\\CN_ARB-main(1)\\3ific.html'